In [21]:
import numpy as np
import tensorflow as tf

In [142]:
tf.reset_default_graph()

x = tf.Variable(np.array([
    0, 0, 7, 0, 0, 0, 3, 0, 2,
    2, 0, 0, 0, 0, 5, 0, 1, 0,
    0, 0, 0, 8, 0, 1, 4, 0, 0,
    0, 1, 0, 0, 9, 6, 0, 0, 8,
    7, 6, 0, 0, 0, 0, 0, 4, 9,
    0, 0, 0, 0, 0, 0 ,0, 0, 0,
    0, 0, 0, 1, 0, 3, 0, 0, 0,
    8, 0, 1, 0, 6, 0, 0, 0, 0,
    0, 0, 0 ,7, 0, 0, 0 ,6, 3
]).reshape(9,9))

idx = tf.tile(
    tf.constant((np.arange(9)+1).reshape(9, 1, 1), tf.int32),
(1, 9, 9))

idxV = tf.tile(
    tf.constant((np.arange(9)+1).reshape(1, 9, 1), tf.int32),
(9, 1, 9))

dom_zero = tf.equal(x, 0)
n = tf.reduce_sum(tf.cast(dom_zero, tf.int32))
dom0 = tf.broadcast_to(dom_zero, (9,9,9))
domH = tf.stack([
    tf.tile(
        tf.reshape(
            tf.logical_not(tf.reduce_any(tf.equal(x, v), 1)),
        (9, 1)),
    (1, 9)) for v in range(1,10)
])
domV = tf.stack([
    tf.reshape(
        tf.tile(
            tf.logical_not(tf.reduce_any(tf.equal(x, v), 0)),
        (9,)),
    (9,9)) for v in range(1,10)
])
domB = tf.reshape(tf.transpose(
    tf.reshape(tf.tile(
        tf.equal(
            tf.transpose(tf.nn.max_pool(
                tf.reshape(
                    tf.stack([
                        tf.cast(tf.equal(x, v), tf.int32)
                            for v in range(1,10)
                    ], axis=2),
                (1,9,9,9)),
                (1,3,3,1),
                (1,3,3,1),
                'VALID',
            ), (3,1,2,0)),
        0), # shape: 9,3,3,1
    (1,1,1,9)), (9,3,3,3,3)),
(0,1,3,2,4)), (9,9,9))
dom = tf.cast(tf.logical_and(
    dom0, tf.logical_and(
        domB, tf.logical_and(
            domH, domV
        )
    )
), tf.int32)
ct0 = tf.cast(tf.equal(tf.reduce_sum(dom, 0), 1), tf.int32)
assign0 = tf.multiply(tf.reduce_sum(tf.multiply(dom, idx), 0), ct0)
ctV = tf.cast(tf.equal(tf.reduce_sum(dom, 1), 1), tf.int32)
assignV = tf.multiply(tf.reduce_sum(tf.multiply(dom, idxV), 1), ctV)
# ctH = tf.cast(tf.equal(tf.reduce_sum(dom, 2), 1), tf.int32)
# assignH = tf.multiply(tf.reduce_sum(lookup, 2), ctH)


init = tf.global_variables_initializer()


In [144]:
with tf.Session() as sess:
    sess.run(init)
    for _ in range(sess.run(n)):
        print(sess.run(assign0))
        print(sess.run(assignV))
        break

[[0 0 0 0 4 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[1 0 0 0 0 0 0 0 6]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 6 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
